In [1]:

from copy import deepcopy

import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import time

In [2]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [3]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
def generate_teacher_data(teacher, num_samples=1000):
    X = torch.randn(num_samples, teacher.fc1.in_features)
    with torch.no_grad():
        y = teacher(X)
    return X, y

In [5]:
class Teacher(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.act = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.init_weights()
        
    def init_weights(self):
        for param in self.parameters():
            param.data.uniform_(-1, 1)
    
    def forward(self, x):
        return self.fc2(self.act(self.fc1(x)))

In [6]:
input_size = 100
hidden_teacher = 50
output_size = 10

In [7]:
teacher = Teacher(input_size, hidden_teacher, output_size)
X, y = generate_teacher_data(teacher)

In [8]:
X.shape, y.shape

(torch.Size([1000, 100]), torch.Size([1000, 10]))

In [9]:
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(X, y)

# train_dataset, val_dataset = random_split(dataset, [0.8, 0.2])

train_loader = DataLoader(dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(dataset, batch_size=128, shuffle=False)

In [10]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28, hidden_size=16):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, 10)
        # self.fc1 = nn.Linear(hidden_size, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc0(x)
        return x

In [22]:
class MLP(nn.Module):
    def __init__(self, input_size=28 * 28, hidden_size=16, output_size=10):
        super(MLP, self).__init__()
        self.fc0 = nn.Linear(input_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, output_size)
        self.act = nn.ReLU()

    def forward(self, x):
        return self.fc1(self.act(self.fc0(x)))

In [23]:
model = MLP(input_size=input_size, hidden_size=hidden_teacher, output_size=output_size)
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0, model.fc1], device=device)

In [24]:
hyperparams = {
    "num_epochs": 512,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.55}, # 1.0 -> all edges, 0.0 -> no edges it isn't true TODO !!!!
    "choose_thresholds_del": {"fc0": 0.2}, # 1.0 -> all edges, 0.0 -> no edges
    "threshold": 1,
    "min_delta_epoch_replace": 64,
    "window_size": 10000,
    "lr": 0.01,
    "delete_after": 2,    
    "task_type": "regression",
    "fully_connected": False,
    "max_to_replace": 900 # None -> no limit
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 512, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.55}, choose_thresholds_del: {'fc0': 0.2}, threshold: 1, min_delta_epoch_replace: 64, window_size: 10000, lr: 0.01, delete_after: 2, task_type: regression, fully_connected: False, max_to_replace: 900"

In [25]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [26]:
wandb.finish()
run = wandb.init(
    project="self-expanding-nets-teacher-student",
    name=f"trash",
    config=hyperparams
)

acc amount,█▅▄▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
len_choose,█▆▂▁▁▁▂
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
n_params over train_time,▁▁▂▂▁▂▂▁▅▆▆▅▇▂▆█▇█▆▇▆█▄▇▃▃▇▇▆▇▆▃▆▃▆▆▃▆▇▆
params amount,▁▁▁▁▅▅██████████████████████████████████
params ratio,▅▄▃▁▄███████████████████████████████████
params to replace amount,▇▄▇▆█▇▅▃▃▃▃▃▃▄▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▁▁▁
train loss,█▇▇▆▅▄▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train time,▁▁▁▁▂▂▂▂▃▃▂▂▂▃▂▃▃▃▃▇▃▃▃▃▃▃▃▃█▃▃▃▃▃▄▃▇▃▃▃
train_time over n_params,▅▅▄▅▅▁▂▂▁▁▁▁▁▂▁█▃▁▃▁▁▁▁▁▂▂▂▁▁▁▃▂▁▃▁▁▁▂▁▁
val accuracy,█▆▆▅▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [27]:
X[0], y[0]

(tensor([ 0.5036, -0.1890, -0.8230,  0.6408, -0.0177, -1.6012, -1.4133, -0.1104,
         -1.2363,  0.4655, -1.2653,  0.0480,  0.2607,  1.1390, -1.0824, -1.0325,
          1.3176,  0.5105, -0.5620,  0.1363, -0.6406, -1.0693, -1.9519, -0.1173,
          0.0434, -0.5631, -2.0348, -1.7682, -0.2879,  0.5706,  1.6687, -0.8882,
          0.4668, -0.6042,  0.1897, -0.4064, -0.1386, -0.0842,  1.3771,  0.4320,
         -0.8417, -0.5110, -1.1281, -1.3884, -0.0073, -1.2156,  0.8475, -0.5152,
         -0.8448,  1.1842,  0.1005, -0.5746,  0.0572,  0.1099, -0.1112, -0.5126,
          0.6286,  0.0367,  0.8207,  0.0929,  0.6649,  1.6473, -0.9514, -2.3191,
         -1.1406,  0.1786,  0.0582,  1.0344, -0.1432, -0.3607,  1.0242, -1.1942,
          0.3042, -0.4627,  0.4357,  0.4721,  0.1969, -1.3440,  0.5754, -0.5774,
         -1.3592, -1.3901, -0.8245, -0.6401, -1.2309,  0.6116, -1.1174, -1.5687,
         -1.5826, -1.1972, -1.3813, -0.6246,  0.6300,  0.9758, -1.6571, -0.6462,
          1.7626,  0.9456, -

In [28]:
criterion = nn.MSELoss()
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, criterion, hyperparams, device)

100%|██████████| 8/8 [00:00<00:00, 280.26it/s]


Epoch 1/512, Train Loss: 318.5893, Val Loss: 296.2576, Val Accuracy: 296.1407


100%|██████████| 8/8 [00:00<00:00, 586.38it/s]


Epoch 2/512, Train Loss: 272.9432, Val Loss: 237.7455, Val Accuracy: 237.6406


100%|██████████| 8/8 [00:00<00:00, 489.27it/s]


...

Epoch 510/512, Train Loss: 7.9165, Val Loss: 7.5878, Val Accuracy: 7.5901


100%|██████████| 8/8 [00:00<00:00, 527.78it/s]


Epoch 511/512, Train Loss: 7.9421, Val Loss: 7.4542, Val Accuracy: 7.4596


100%|██████████| 8/8 [00:00<00:00, 302.54it/s]


Epoch 512/512, Train Loss: 7.9012, Val Loss: 7.4363, Val Accuracy: 7.4402
